# Assignment 4: Neural Network

## NeuralNetwork Class

In [1]:
import numpy as np
from scipy.optimize import fmin_cg


class NeuralNetwork:
    def __init__(self, layers, X, y):
        # layers is a list consisting of no. of neurons in each layer
        # bias neuron is NOT included
        # bias neuron is taken care of in init_weights_randomized method
        # e.g.: layers = [400, 16, 16, 10]
        self.layers = layers
        self.L = len(layers)
        self.X_not_biased = X
        self.optimum_weights = None

        self.m = self.X_not_biased.shape[0]
        self.n = self.X_not_biased.shape[1]
        self.y = y.reshape(self.m, 1)

        bias = np.ones((self.m, 1))
        self.X_biased = np.append(bias, self.X_not_biased, axis=1)
        self.weights = self.init_weights_randomized()
        # self.weights_vector = self.unroll(self.weights)

    def train(self):
        thetavec = self.unroll_list_of_matrices(self.weights)
        xvec = self.unroll_X_matrix(self.X_not_biased)

        result = minimize(method='CG', fun=self.cost, jac=self.back_propagate, x0=thetavec, args=(xvec, self.y, 1), options={'maxiter': 50})
        rows = result.x.size
        self.optimum_weights = self.roll_weights(result.x.reshape(rows, 1))

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def sigmoid_prime(self, z):
        # np.multiply(): element-wise multiplication
        return np.multiply(self.sigmoid(z), 1 - self.sigmoid(z))

    def init_weights_randomized(self):
        # if we have L layers, we are going to have L-1 weight matrices
        # this method returns a 'list' of weight matrices
        result = []
        for i in range(0, self.L - 1):
            matrix = np.random.random((self.layers[i+1], 1 + self.layers[i]))
            matrix = 2 * matrix - 1
            result.append(matrix)
        return result

    def unroll_list_of_matrices(self, matrices):
        mylist = []
        size = 0
        for i in range(len(matrices)):
            size += matrices[i].shape[0] * matrices[i].shape[1]
            mylist.append(list(matrices[i].ravel()))
        for i in range(1, len(mylist)):
            mylist[0] = mylist[0] + mylist[i]
        return np.array(mylist[0]).reshape(size, 1)

    def unroll_X_matrix(self, X_matrix):
        size = X_matrix.shape[0] * X_matrix.shape[1]
        return X_matrix.reshape(size, 1)

    def roll_weights(self, weights_vector):
        # this function gets the unrolled-vector and returns matrix(s)
        result = []
        start = 0
        # just to make sure it's a vector
        weights_vector = weights_vector.reshape(weights_vector.size, 1)
        for i in range(self.L - 1):
            shape = (self.layers[i+1], 1 + self.layers[i])
            no = shape[0] * shape[1]
            end = no + start
            cur = weights_vector[start:end, ...].reshape(shape)
            start = end
            result.append(cur)
        return result

    def roll_X(self, X_vector):
        return np.array(X_vector).reshape(self.m, self.n)

    def feed_forward(self, x_vector, weights):
        # this function gets a specific vector, and returns activations ...
        # of neural network to the vector
        # x_vector is biased INSIDE this function
        # so there is NO NEED to add the bias before passing to this function
        zs_activations = [(x_vector, x_vector)]
        vector = x_vector.reshape(self.n, 1)
        cur_vector = vector
        bias = np.array([[1]])
        for weight_matrix in weights:
            i = weights.index(weight_matrix)
            # add the bias:
            cur_vector = np.append(bias, cur_vector)
            cur_vector = cur_vector.reshape(1 + self.layers[i], 1)
            z = np.matmul(weight_matrix, cur_vector)
            a = self.sigmoid(z)
            cur_vector = a
            zs_activations.append((z, a))

        return zs_activations

    def refine_y(self, y_value):
        # this funcion receives a value of y ...
        # and returns a vector which has only 1 nonzero value
        # dimension of the returned vector is equal to number of classes
        result = np.zeros((self.layers[self.L - 1], 1))
        result[y_value][0] = 1
        return result

    def cost(self, weights_vector, X_vector, y, lmd):
        X = self.roll_X(X_vector)
        weights = self.roll_weights(weights_vector)
        cost = 0
        for i in range(self.m):
            x_vector = X[i].reshape((self.n, 1))
            activations = self.feed_forward(x_vector, weights)
            hypothesis = activations[-1][1]
            my_y = self.refine_y(y[i][0])
            cost += np.matmul(my_y.T, np.log(hypothesis))
            cost += np.matmul((1 - my_y).T, np.log(1 - hypothesis))
        cost *= (-1 / self.m)

        # regularization term
        reg = 0
        if lmd != 0:
            for weight_matrix in weights:
                reg += np.sum(np.multiply(weight_matrix, weight_matrix))
            reg *= lmd / (self.m * 2)

        return cost + reg

    def back_propagate(self, weights_vector, X_vector, y, lmd):
        weights_matrices = self.roll_weights(weights_vector)
        X = self.roll_X(X_vector)
        deltas = [None for i in range(0, self.L)]
        # deltas[0] is never used
        Deltas = []
        Ds = []
        # Deltas[L-1] is never used
        for i in range(0, self.L - 1):
            shape = (self.layers[i + 1], 1 + self.layers[i])
            Deltas.append(np.zeros(shape))
            Ds.append(np.zeros(shape))

        for i in range(0, self.m):
            x_vector = X[i].reshape(self.n, 1)
            zs_activations = self.feed_forward(x_vector, weights_matrices)
            deltas[-1] = zs_activations[-1][1] - self.refine_y(y[i][0])
            for j in range(self.L - 2, 0, -1):
                deltas[j] = np.matmul(weights_matrices[j].T, deltas[j+1])
                # elementwise multiplication:
                deltas[j][1:, ...] *= self.sigmoid_prime(zs_activations[j][0])
                # remove the bias:
                deltas[j] = deltas[j][1:, ...]

            for j in range(0, self.L - 1):
                activation = zs_activations[j][1]
                bias = np.array([[1]])
                activation = np.append(bias, activation)
                activation = activation.reshape(1 + self.layers[j], 1)
                Deltas[j] += np.matmul(deltas[j+1], activation.T)

        for i in range(0, self.L-1):
            Ds[i] = Deltas[i] / self.m
            if lmd != 0:
                Ds[i][..., 1:] += (lmd / self.m) * weights_matrices[i][..., 1:]

        Deltas = self.unroll_list_of_matrices(Deltas)

        # for scipy, we need 1-D arrays as gradient
        return np.ndarray.flatten(Deltas)

    def get_accuracy(self):
        corrects = 0
        for i in range(0, self.m):
            x_vec = self.X_not_biased[i].reshape(self.layers[0], 1)
            final_activation = self.feed_forward(x_vec, self.optimum_weights)[-1][1]
            mylist = list(final_activation.ravel())
            maximum = max(mylist)
            idx = mylist.index(maximum)
            if idx == self.y[i][0]:
                corrects += 1
        return 100 * corrects / self.m


## Training and accuracy

In [3]:
import scipy.io as sio


X = sio.loadmat('ex3data1.mat')['X']
y = sio.loadmat('ex3data1.mat')['y']
for i in range(y.shape[0]):
    if y[i][0] == 10:
        y[i][0] = 0


nn = NeuralNetwork([400, 25, 10], X, y)
nn.train()
print('Accuracy: ', nn.get_accuracy())


         Current function value: 0.819525
         Iterations: 45
         Function evaluations: 529
         Gradient evaluations: 529
Accuracy:  94.76
